In [180]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from numpy import sqrt

In [181]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [182]:
train = pd.read_csv('/content/drive/My Drive/train_new.csv')

In [214]:
test = pd.read_csv('/content/drive/My Drive/test_new.csv')

In [184]:
train.head()

,SalePrice,PID,Lot Frontage,Lot Area,Street,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,...,Full Bath,Half Bath,Bedroom AbvGr,TotRms AbvGrd,Gr Liv Area,Functional,Screen Porch,Pool Area,Yr Sold,Sale Type
0,159000,531363010,80.0,9605,Pave,SawyerW,1Fam,1Story,7,6,...,1,1,3,6,1218,Typ,0,0,2009,WD
1,271900,906203120,90.0,14684,Pave,SawyerW,1Fam,1Story,7,7,...,2,0,3,7,2196,Typ,0,0,2009,WD
2,137500,916176030,NaN,14375,Pave,Timber,1Fam,SLvl,6,6,...,1,0,3,7,1344,Typ,233,0,2009,COD
3,248500,528180130,48.0,6472,Pave,NridgHt,TwnhsE,1Story,9,5,...,2,0,2,6,1456,Typ,0,0,2009,WD
4,167000,528290030,61.0,9734,Pave,Gilbert,1Fam,SLvl,7,5,...,2,1,3,7,1374,Typ,0,0,2009,WD


In [185]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2197 entries, 0 to 2196
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SalePrice      2197 non-null   int64  
 1   PID            2197 non-null   int64  
 2   Lot Frontage   1835 non-null   float64
 3   Lot Area       2197 non-null   int64  
 4   Street         2197 non-null   object 
 5   Neighborhood   2197 non-null   object 
 6   Bldg Type      2197 non-null   object 
 7   House Style    2197 non-null   object 
 8   Overall Qual   2197 non-null   int64  
 9   Overall Cond   2197 non-null   int64  
 10  Year Built     2197 non-null   int64  
 11  Roof Style     2197 non-null   object 
 12  Heating        2197 non-null   object 
 13  Central Air    2197 non-null   object 
 14  Electrical     2196 non-null   object 
 15  Full Bath      2197 non-null   int64  
 16  Half Bath      2197 non-null   int64  
 17  Bedroom AbvGr  2197 non-null   int64  
 18  TotRms A

In [303]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

train_numeric = train.select_dtypes(include=['int64', 'float64'])
scaler = StandardScaler()
train_numeric_scaled = scaler.fit_transform(train_numeric)
imputer = KNNImputer(n_neighbors=5)
train_numeric_imputed = imputer.fit_transform(train_numeric_scaled)
train_numeric_imputed = pd.DataFrame(train_numeric_imputed, columns=train_numeric.columns)
train_numeric_imputed = scaler.inverse_transform(train_numeric_imputed)
train[train_numeric.columns] = train_numeric_imputed


In [304]:
test_numeric = test.select_dtypes(include=['int64', 'float64'])
scaler = StandardScaler()
test_numeric_scaled = scaler.fit_transform(test_numeric)
imputer = KNNImputer(n_neighbors=5)
test_numeric_imputed = imputer.fit_transform(test_numeric_scaled)
test_numeric_imputed = pd.DataFrame(test_numeric_imputed, columns=test_numeric.columns)
test_numeric_imputed = scaler.inverse_transform(test_numeric_imputed)
test[test_numeric.columns] = test_numeric_imputed

In [193]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2197 entries, 0 to 2196
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SalePrice      2197 non-null   float64
 1   PID            2197 non-null   float64
 2   Lot Frontage   2197 non-null   float64
 3   Lot Area       2197 non-null   float64
 4   Street         2197 non-null   object 
 5   Neighborhood   2197 non-null   object 
 6   Bldg Type      2197 non-null   object 
 7   House Style    2197 non-null   object 
 8   Overall Qual   2197 non-null   float64
 9   Overall Cond   2197 non-null   float64
 10  Year Built     2197 non-null   float64
 11  Roof Style     2197 non-null   object 
 12  Heating        2197 non-null   object 
 13  Central Air    2197 non-null   object 
 14  Electrical     2196 non-null   object 
 15  Full Bath      2197 non-null   float64
 16  Half Bath      2197 non-null   float64
 17  Bedroom AbvGr  2197 non-null   float64
 18  TotRms A

In [305]:
numeric_cols = train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = train.select_dtypes(include=['object']).columns
train_encoded = pd.get_dummies(train, columns=categorical_cols)

In [306]:
X = train_encoded.drop(['SalePrice'], axis = 1)
y = train_encoded['SalePrice']

In [307]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Fit RandomForestRegressor to get feature importances
categorical_cols = X.select_dtypes(include=['object']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('dummify', OneHotEncoder(), categorical_cols)
    ],
    remainder='passthrough'
)

X_transformed = preprocessor.fit_transform(X)
feature_names = preprocessor.get_feature_names_out()
X_transformed = pd.DataFrame(X_transformed, columns=feature_names)

forest = RandomForestRegressor(random_state=0)
forest.fit(X_transformed, y)

importances = forest.feature_importances_

sorted_indices = np.argsort(importances)[::-1]
for idx in sorted_indices:
    print(f"{feature_names[idx]}: {importances[idx]}")

remainder__Overall Qual: 0.6622108769921498
remainder__Gr Liv Area: 0.15043143170269463
remainder__Lot Area: 0.03832893690239409
remainder__Year Built: 0.03752213713725956
remainder__PID: 0.021103592637618074
remainder__Lot Frontage: 0.015904518864705983
remainder__Bedroom AbvGr: 0.007715022757212784
remainder__Full Bath: 0.0071676856212756915
remainder__Screen Porch: 0.006582070379110832
remainder__Overall Cond: 0.006176880684684441
remainder__TotRms AbvGrd: 0.005344787176533621
remainder__Yr Sold: 0.004380785318875947
remainder__House Style_1Story: 0.004035120096204659
remainder__Pool Area: 0.0024851448289255184
remainder__House Style_2Story: 0.0024600557069447567
remainder__Half Bath: 0.001987861817829077
remainder__Roof Style_Hip: 0.001787195514350541
remainder__Neighborhood_Edwards: 0.0016579177377642595
remainder__Central Air_Y: 0.001569293904939967
remainder__Central Air_N: 0.001328723267242955
remainder__Roof Style_Gable: 0.0012818419219018191
remainder__Neighborhood_NoRidge: 0

In [308]:
X = train[['Overall Qual', 'Gr Liv Area', 'Lot Area', 'Year Built', 'Lot Frontage', 'Bedroom AbvGr', 'Full Bath', 'Neighborhood', 'Overall Cond', 'Screen Porch','House Style', 'Bldg Type']]
y = train['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [309]:
ct = ColumnTransformer(
    [
        ("num", Pipeline([("scale", StandardScaler())]),
         make_column_selector(dtype_include=np.number)),
        ("cat", Pipeline([("enc", OneHotEncoder(handle_unknown='ignore', sparse_output=False))]),
         make_column_selector(dtype_include='object'))
    ]
)

elastic_pipeline = Pipeline([
    ('preprocessing', ct),
    ('elasticnet', ElasticNet(alpha=0.7, l1_ratio=0.3))
])
elastic_pipeline_fitted = elastic_pipeline.fit(X_train, y_train)

In [310]:
y_pred = elastic_pipeline_fitted.predict(X_test)
log_y_test = np.log(y_test)
log_y_pred = np.log(y_pred)
rmse = np.sqrt(mean_squared_error(log_y_test, log_y_pred))
rmse

0.1728978933432756

In [311]:
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [312]:
from sklearn.svm import SVR

svr_pipeline = Pipeline([
    ('preprocessing', ct),
    ('svr', SVR())
])

svr_pipeline_fitted = svr_pipeline.fit(X_train, y_train)

In [313]:
y_pred = svr_pipeline_fitted.predict(X_test)
log_y_test = np.log(y_test)
log_y_pred = np.log(y_pred)
rmse = np.sqrt(mean_squared_error(log_y_test, log_y_pred))
rmse

0.3790474229679823

In [314]:
X = train[['Overall Qual', 'Gr Liv Area', 'Lot Area', 'Year Built', 'Lot Frontage', 'Bedroom AbvGr', 'Full Bath', 'Neighborhood', 'Overall Cond', 'Screen Porch', 'House Style', 'Bldg Type']]
y = train['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [315]:
from sklearn.ensemble import GradientBoostingRegressor

gbr_pipeline = Pipeline([
    ('preprocessing', ct),
    ('gradient_boosting', GradientBoostingRegressor())
])

gbr_pipeline_fitted = gbr_pipeline.fit(X, y)

In [316]:
y_pred = gbr_pipeline_fitted.predict(X_test)
log_y_test = np.log(y_test)
log_y_pred = np.log(y_pred)
rmse = np.sqrt(mean_squared_error(log_y_test, log_y_pred))
rmse

0.11451850095452076

In [274]:
test.head()

,PID,Lot Frontage,Lot Area,Street,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,...,Full Bath,Half Bath,Bedroom AbvGr,TotRms AbvGrd,Gr Liv Area,Functional,Screen Porch,Pool Area,Yr Sold,Sale Type
0,907135180,60,8070,Pave,CollgCr,1Fam,1Story,4,5,1994,...,1,0,3,5,990,Typ,0,0,2007,WD
1,528181040,40,6792,Pave,NridgHt,TwnhsE,1Story,7,5,2005,...,2,0,2,6,1368,Typ,0,0,2006,New
2,528175010,44,6371,Pave,NridgHt,TwnhsE,1Story,7,5,2009,...,2,0,2,6,1358,Typ,0,0,2010,New
3,531379030,70,8304,Pave,SawyerW,1Fam,2Story,6,5,1997,...,2,1,3,7,1837,Typ,0,0,2006,WD
4,923275090,37,6951,Pave,Mitchel,1Fam,1Story,5,5,1984,...,1,0,3,5,923,Typ,0,0,2008,WD


In [317]:
predicted_sale_prices = gbr_pipeline_fitted.predict(test)

In [318]:
predicted_prices = pd.DataFrame({
    'PID': test['PID'],
    'SalePrice': predicted_sale_prices
})

In [319]:
predicted_prices.to_csv('/content/drive/My Drive/price_predictions.csv', index=False)